## Class 2: Linear Algebra and Stats in Python (continued)

### Outline of this notebook
* **getting some data**
* **mean, median, mode**
* **variance and standard deviation**
* **covariance and correlation**
* **histograms**
* **scatter plots**
* **random variables and distributions**
* **the normal distribution - useful properties**
* **the exponential distribution - useful properties**
* **important properties of empirical distributions - skew and kurtosis**

When you look at any collection of data, you want to be able to look at just a few numbers that provide a useful indication of how the whole collection behaves.

Statistics is the study of those small sets of numbers that provide a useful indication of how any collection of numbers (arbitrarily large) behavies.

We call these collections of numbers **distributions** and the smaller set of numbers that describe them as the **properties of distributions**.

In [ ]:
from __future__ import print_function, unicode_literals, division
import numpy as np
from scipy import stats
import requests
import csv

%matplotlib inline
import seaborn as sns
sns.set(rc={"figure.figsize": (10, 6)})

Ok, lets get some data that we can actually work with:

In [ ]:
#Get iris dataset from online
r = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data') 

iris_data = [row.split(",") for row in r.iter_lines()]

print("First five entries in iris_data:")
for row in iris_data[0:5]:
    print(row)
print("Last five entries in iris_data:")
for row in iris_data[-5:]:
    print(row)

We are going to have to clean it up a bit:

In [ ]:
#Remove last line as it is empty
iris_data_cleaned = iris_data[:-1]

#Put last element in each entry into separate list
iris_names = [row[-1] for row in iris_data_cleaned]
print("First five entries in iris_names:")
for row in iris_names[0:5]:
    print(row)

#Put all elements but last in each entry into values list
iris_data_values = [row[:-1] for row in iris_data_cleaned]
print("First five entries in iris_data_cleaned:")
for row in iris_data_values[0:5]:
    print(row)

#Convert values into floats
iris_data_values_float = [[float(value) for value in row] for row in iris_data_values]
print("First five entries in iris_data_values_float:")
for row in iris_data_values_float[0:5]:
    print(row)

#Convert list of list of floats into matrix
iris_data_final = np.array(iris_data_values_float)
print("First five entries in iris_data_final:")
for row in iris_data_final[0:5]:
    print(row)

`iris_data_final` contains the properly formatted numeric data, so we will be using that to illustrate all of the statistics going forward.

### mean, median, mode

The **mean** is the arithmetic average of a group of values, found by dividing the total of all values by the number of values.

The **median** is the middle value in a group of values, found by ordering the values from smallest to largest and locating the one that occurs in the middle. If the size of the group is even, it is found by averaging the middle two values.

The **mode** is the value that occurs most often in a group of values, and is found by counting the frequency of every distinct value in the group and outputting the one that occurs most frequently.

Let's compute each of these properties in turn on the columns in the `iris_data_final` matrix:

In [ ]:
print("Shape of the data (rows,columns):",iris_data_final.shape)
print("Mean of each column:",iris_data_final.mean(axis=0))
#must pass in an axis argument, otherwise will compute mean of entire matrix
print("Median of each column:",np.median(iris_data_final,axis=0))
print("Mode of each column:",stats.mode(iris_data_final)) 
#implied axis=0 for this function

### variance and standard deviation

It is useful to get a sense of how the values in a dataset are spread about the average or the central value in that dataset. **Variance** and **standard deviation** are values that quantify that spread.

The **variance** of a set of values, **$\sigma^2$**, is the average of the square of the difference of the values in the dataset from the dataset's mean:

\begin{align}
\sigma^2 = \frac{\sum_{i=1}^{n}(x_i - \mu)^2} {n}\
\end{align}

We square the differences to preserve the magnitude of the difference only (otherwise we would be adding and subtracting values, and lose the average magnitude). So, the variance is not on the same scale as the values themselves:

In [ ]:
print("Variance of each column:\n",iris_data_final.var(axis=0))
#must pass in an axis argument, otherwise will compute mean of entire matrix

The **standard deviation**, $\sigma$, is the square root of the variance:

\begin{align}
\sigma = \sqrt\frac{\sum_{i=1}^{n}(x_i - \mu)^2} {n}\
\end{align}

We use the **standard deviation** much more regularly than the **variance** because it is on the same scale as the original data:

In [ ]:
print("Standard deviation of each column:\n",iris_data_final.std(axis=0))
#must pass in an axis argument, otherwise will compute mean of entire matrix

### covariance and correlation

So far, we've been looking at statistical properties of individual collections of data (in our case, columns of the `iris_data_final` matrix) by themselves. However, it is also useful to quantify how two collections of data vary with each other. **Covariance** and **correlation** measure this exact quantity.

**Covariance**, like the variance, is a measure of spread, however it also measures how closely two datasets track each other. That is, **covariance** describes both how far the variables are spread out, and the nature of their relationship, whether they tend to increase and decrease together, or whether they do not track each other at all. Like **variance**,  the **covariance** is a squared quantity (again, to simply measure the magnitude of the spread and to avoid subtracting numbers from each other) so it is not on the same scale as the mean (and the **covariance** of different pairs of variables can have completely different scales):

\begin{align}
Cov(x,y) = \frac{\sum_{i=1}^{n}(x_i - \mu_x)(y_i - \mu_y)} {n}\
\end{align}

When we look at the covariance of a dataset, we compute what is called a **covariance matrix**.

This matrix is symmetric and tells us what all the paired covariance values are for every pair of rows (or columns) in our original matrix:

In [ ]:
#have to take the transpose of iris_data_final, because np.cov computes row-based covariances
#so if we take the transpose, our columns become our rows!
iris_data_final_column_cov = np.cov(iris_data_final.T)
print("Original iris_data_final matrix shape:",iris_data_final.shape)
print("Columnar covariance matrix:\n",iris_data_final_column_cov,
      iris_data_final_column_cov.shape)

**Correlation** is sort of like **standard deviation** generalized to pairs of datasets.

Really, the **correlation** is a **covariance** scaled by each dataset's **standard deviation**, so that it can only take on values from -1 to +1. This allows us to compare two pairs of variables and quickly tell if one pair of datasets is more related than an other (since both values are scaled to the same range!):

\begin{align}
r(x,y) = \frac{cov(x,y)} {\sigma_x\sigma_y}\
\end{align}

Or this:

\begin{align}
r(x,y) = \frac{\frac{\sum_{i=1}^{n}(x_i-\mu_x)(y_i-\mu_y)} {n}} {\sigma_x\sigma_y}\
\end{align}

A positive correlation indicates the sets of values change together (when one increases or decreases the other does the same). A negative correlation indicates that the variables change in opposite directions (when one increases the other decreases).

Just like the **covariance**, when we compute **correlations** for some dataset, we really compute a **correlation matrix** over all of the distinct sets of values in the dataset. The **correlation matrix** is symmetric, just like the **covariance matrix**:

In [ ]:
#again, we have to take the transpose because otherwise we will get row-wise correlations.
iris_data_final_column_corr = np.corrcoef(iris_data_final.T)
print("Original iris_data_final matrix shape:",iris_data_final.shape)
print("Columnar covariance matrix:\n",iris_data_final_column_corr,
      iris_data_final_column_corr.shape)

Lets take a look at the correlation matrix visually too:

In [ ]:
sns.heatmap(iris_data_final_column_corr)
print("The correlation matrix for the iris dataset:")

#### Exercise Time!
* Compute the row-based correlation matrix for `iris_data_final`.
* Visualize the correlation matrix as a heatmap. Notice anything?
* I've loaded in vertebral_data into a `NumPy` matrix called `vertebral_values_final` for you below:
    * Compute the mean of each column. 
    * Compute the median of each column.
    * Compute the mode of each column.
    * What would it mean if the mean and the median for a given column are very far apart?
    * What is useful about knowing the mode?
    * Compute the variance and standard deviation of each column.
    * Generate the columnar covariance and correlation matrices for this matrix.
    * Do any columns appear to change together (based on their covariances/correlations)?
    * What conclusions can we draw from these column-based statistics?
    * If we had computed all of the row-based statistics here, what would their interpretation be?

In [ ]:
with open('../data/vertebral_column_2_categories.dat', 'r') as f:
    vertebral_data = [row for row in csv.reader(f)]

vertebral_data_parsed = [str.split(row[0],' ') for row in vertebral_data]
vertebral_names = [row[-1] for row in vertebral_data_parsed]
vertebral_values = [row[:-1] for row in vertebral_data_parsed]
vertebral_values_float = [[float(value) for value in row] for row in vertebral_values]
vertebral_values_final = np.array(vertebral_values_float)
print("vertebral_values_final = \n", vertebral_values_final)

In [ ]:
#This is where you write the code for your exercise.
#Make sure you execute the cell just above this one so that vertebral_values_final is populated.
pass

### histograms

A regular histogram is a very powerful, simple way to get a sense of the kinds of values a single set of data is comprised of. 

Let's take a look at the histogram of values for the first column in `iris_data_final`: 

In [ ]:
sns.distplot(iris_data_final[:,0],kde=False,bins=15)
print("First column of the iris dataset:")

The **x-axis shows us the actual range of values that the column takes on**. It looks like the first column ranges from just over 4 to just under 8.

The **y-axis is a frequency count of the number of rows in the column that fall into each vertical bin**. So, the first bin (furthest to the left), which ranges from ~4.25 to ~4.75 contains 11 values.

We can also make **cumulative history histograms**, which give us a graphical way to see what fraction of the datapoints are below or above a certain value:

In [ ]:
sns.distplot(iris_data_final[:,0], hist_kws={"cumulative":True},kde_kws={"cumulative":True})
print("Cumulative frequency plot of 1st column:")

### scatter plot

When you want to visualize how two columns vary together, you can use a **scatter plot**. 

A scatter plot simply plots the value for each column per-row for the two columns you've chosen.

Every dot in the scatter plot is an individual row. 

Let's generate the scatter plot for the first two columns in the iris dataset:

In [ ]:
sns.plt.scatter(iris_data_final[:,0], iris_data_final[:,1])

You can combine both a **scatter plot** and a **histogram** for a pair of columns very effectively using a `jointplot`:

In [ ]:
sns.jointplot(iris_data_final[:,1], iris_data_final[:,0],stat_func=None)
print("Scatter plot with histograms for the first two columns in the iris dataset:")

### Exercise Time!
* Using the `vertebral_values_final` dataset:
    * Compute the column-wise correlation matrix
    * Visualize the scatter plot for two columns that are positively correlated
    * Visualize the scatter plot for two columns that are negatively correlated (anti-correlated)

In [ ]:
pass

## random variables and distributions 

A **random variable** is a variable whose value is subject to variations due to chance. A random variable can take on a set of possible different values (similarly to other mathematical variables), each with an associated probability. [random variable wiki](https://en.wikipedia.org/wiki/Random_variable)

There are two kinds of random variables: **discrete** and **continuous**

You know about **discrete** random variables, because you've flipped a coin (at some point in your life).

For an unbiased coin, the random variable can take on the values **heads** or **tails**, each with probability $p= 0.5$ but you don't know when it will take on the value. Let's say we flipped a coin ten times, how many heads will we get?:

In [ ]:
n_trials = 10
prob_heads = 0.5
num_heads = np.random.binomial(n_trials, prob_heads)
print("Num heads:",num_heads)
print("Fraction heads:",float(num_heads)/n_trials) #this should be different for most people

All of us should have different numbers! That's because you can only talk about the probabilities, not the outcomes (you only know the outcome of an experiment after it happens).

What about **continuous** random variables?

A **continuous random variable** is a random variable where the data can take infinitely many values.

Think about how long it takes you to make your bed every morning. The experiment "how long it take me to make my bed" is a **continuous random variable** because it can take one of an infinitely many distinct times (30 seconds, 2.54 minutes, 0 minutes because he doesnt wake up on time and runs to work, etc.).

Let's look at the amount of time it took me to make his bed over the last week:

In [ ]:
bed_making_minutes = np.random.exponential(size=7) 
## This is generating 7 experiments from a specific probability distribution
print(bed_making_minutes)
# Everyone's values should be different, compare with your neighbor to make sure!

### Exercise Time!!!

* Generate 100 coin flips, what fraction of them come up heads?
* Generate 1,000 coin flips, what fraction of them come up heads?
* Generate 100,000 coin flips, what fraction of them come up heads? Notice a pattern?
* Generate 100 bed making experiments, whats the average bed making time?
* Generate 1,000 bed making experiments, whats the average bed making time?
* Generate 100,000 bed making experiments, whats the average bed making time? Notice a pattern?

A **probability distribution** is a table or an equation that links each outcome of a statistical experiment with its probability of occurrence. [probability distribution wiki](https://en.wikipedia.org/wiki/Probability_distribution)

That is, it is a distribution of probabilities over the values that a **random variable** can take on.

There are lots of different kinds of distributions that statisticians have identified and studied. We will talk about just a few of them.

**All of data science boils down to trying to find/capture differences between probability distributions across classes of objects, and combining those differences across many distributions into aggregate predictions.**

### the binomial distribution

The **binomial distribution** is a distribution over discrete values. It has two parameters, `n` and `p`, and is the discrete probability distribution of the number of successes in a sequence of n independent yes/no experiments, each of which yields success (yes) with probability `p`.

A success/failure experiment is also called a Bernoulli experiment or Bernoulli trial; when n = 1 (a single coin flip), the binomial distribution is a Bernoulli distribution  [binomial distribution wiki](https://en.wikipedia.org/wiki/Binomial_distribution)

So, heres the approximate probability distribution of the number of heads in 10 coin flips.

We are going to generate the distribution by simulating 10,000 experiments where we flip a coin 10 times:

In [ ]:
n_trials_binomial=1000
binomial_data = np.random.binomial(n_trials, prob_heads,n_trials_binomial)
sns.plt.hist(binomial_data,normed=True,alpha=0.5,bins=10)
print("A binomial distribution for the number of heads in 10 coin flips:")

### the normal distribution

The normal distribution is the most important distribution in all of statistics. It is a continuous probability distribution, unlike the binomial, which is discrete.

It's super important because as datasets become larger and larger and larger, they tend to look more and more like the normal distribution.

The normal distribution is fully specified by 2 parameters, $\mu$ (its mean) and $\sigma$ (its standard deviation).

Nothing in the world actually perfectly mirrors a normal distribution, because its an idealized, mathematical object. It is simply a theoretical approximation of the kinds of things we tend to see in the world. However, there are many datasets that behave very similarly to a normal distribution. [normal distribution wiki](https://en.wikipedia.org/wiki/Normal_distribution)

In [ ]:
mu = 0
sigma = 1
n_samples = 100000
normal_data = np.random.normal(mu, sigma, n_samples)
sns.distplot(normal_data)
print("100000 values sampled from the normal distribution with mean=0 standard deviation=1:")

In [ ]:
sns.distplot(normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
print("CDF of normal distribution:")

Altering the $\mu$ of the distribution alters its location:

In [ ]:
negative_mu = -3
positive_mu = 3
negative_mu_normal_data = np.random.normal(negative_mu, sigma, n_samples)
positive_mu_normal_data = np.random.normal(positive_mu, sigma, n_samples)
sns.distplot(negative_mu_normal_data)
sns.distplot(positive_mu_normal_data)
sns.distplot(normal_data)

print("100000 values sampled from three normal distributions with " +
      "different mean=-3,0,5 and same standard deviation= 1:")

In [ ]:
sns.distplot(negative_mu_normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(positive_mu_normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
print("CDF of three normal distributions with different mean=-3,0,5 " +
      "and same standard deviation= 1:")

Altering the $\sigma$ of the distribution alters its width:

In [ ]:
larger_sigma = 3
largest_sigma = 5
#Generate 2 collections of 100,000 samples from each distribution. 
larger_sigma_normal_data = np.random.normal(mu, larger_sigma, n_samples)
largest_sigma_normal_data = np.random.normal(mu, largest_sigma, n_samples)
#plot all 3 distributions on the same plot
sns.distplot(largest_sigma_normal_data)
sns.distplot(larger_sigma_normal_data)
sns.distplot(normal_data)
print("100,000 values sampled from three normal distributions with "
      "the same mean = 0 but different standard deviations = 1,3,5:")

In [ ]:
sns.distplot(largest_sigma_normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(larger_sigma_normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(normal_data, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.plt.xlim((-10,10))
print("CDF for three normal distributions with the same mean = 0 " 
      "but different standard deviations = 1,3,5:")

### the exponential distribution

The **exponential distribution** is another very common distribution (and the one we used for my bed-making example). In stats-speak, it is the probability distribution that describes the time between events in a Poisson process, i.e. a process in which events occur continuously and independently at a constant average rate. [exponential distribution wiki]()

An example exponential distribution would describe the time between airplanes landing and taking off at LaGuardia airport during hours that the airport is operating.

The exponential distribution is interesing because it has the property known as being **memoryless**. This means that knowing the history of the distribution has no effect on being able to predict what will happen next.

If the average time between airplane landing/takeoff is 125s, but for the past 5 times it was 30s, 42s, 50s, 10s, and 40s, your best guess about the next successive take/off landing time difference is still 125s (there are no hot streaks in airplane landings).

In [ ]:
e = np.random.exponential(size=n_samples)
sns.plt.hist(e,bins=100,normed=True)
print("The standard exponential distribution:")

In [ ]:
sns.distplot(e, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.plt.xlim((0,11))
print("CDF of exponential distribution:")

In [ ]:
larger_beta = 3
largest_beta = 5
e2 = np.random.exponential(scale = larger_beta, size=n_samples)
e3 = np.random.exponential(scale = largest_beta, size=n_samples)
sns.plt.hist(e3,bins=100,alpha=0.5,hold=True)
sns.plt.hist(e2,bins=100,alpha=0.5,hold=True)
sns.plt.hist(e,bins=100,hold=True)
print("100000 values sampled from three exponential distributions with beta=1,3,5:")

In [ ]:
sns.distplot(e, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(e2, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.distplot(e3, hist_kws={"cumulative":True},kde_kws={"cumulative":True})
sns.plt.xlim((0,20))
print("CDF of three exponential distributions with beta=1,3,5")

**FYI:** $\beta$ is the same as $\frac {1} {\lambda}$ for the other common parameterization of the exponential distribution. 

I like to use $\beta$ instead of $\lambda$ because $\beta$ is much more interpretable.

It is simply the rate/mean/standard deviation of the distribution. $\beta$ is also known as the **survival parameter** of the distribution, because if the distribution is used to model a process that takes a certain time to occur, it is the mean **survival** time of that process.

### the uniform distribution

The final distribution we will talk about is the **uniform distribution**. It simply describes cases where all values have the exact same frequency. It is a useful distribution because it is used for unbiased sampling (in cases where sampling is necessary).

Here's an example uniform distribution over the values between 0 and 1:

In [ ]:
uni = np.random.uniform(size=n_samples)
sns.plt.hist(uni,bins=100,alpha=0.5,hold=True,normed=True)
print("The uniform distribution:")

### Skewness

We are now going to talk about shape properties of all probability distributions. These are the two most common properties that

**Skewness** is a measure of how "skewed" a distribution is. Right, what is "skew"? Really, it measures how asymmetrical a given distribution is around its mean value, and can only be computed for continuous distributions. [skewness wiki](https://en.wikipedia.org/wiki/Skewness)

Here are two examples of skewness, demonstrating positive and negative skew:

![Skewness](../images/skewness_example.png)

Hopefully, you can tell that if these two examples were completely un-skewed, then they would follow the grey dotted lines for the distributions. Positive and negative skew is in reference to the tails of the distributions. **Positive skew** means that the right tail (the part greater than the mean) is bigger than it would be without skew. **Negative skew** means that the left tail (the part of the distribution less than the mean) is bigger than it would be without skew.

### Kurtosis

**Krutosis** is a measure of the "peakedness" of a given probability distribution and can only be measured for continuous distributions. [kurtosis wiki](https://en.wikipedia.org/wiki/Kurtosis)

![Kurtosis](../images/kurtosis_example.png)